In [1]:
import os
import openai
import langchain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

llm = "gpt-3.5-turbo"
chat = ChatOpenAI(temperature=0.0, model=llm)

# Completion function
def get_completion(prompt, model=llm, client=openai):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [22]:
import pandas as pd

inventory_path = "inventory.csv"
orders_path = "orders.csv"

inventory_df = pd.read_csv(inventory_path)
orders_df = pd.read_csv(orders_path)

In [33]:
# Let's take what we need from the orders
projected_quantity_prompt = f""""
Task:
Calculate the projected sales quantity for each product collection for the next month based on the last three months' sales data from orders_df.
Detailed Steps:

Access the DataFrame: <<<{orders_df}>>>

Open orders_df, which contains sales data including purchase_date and quantity for various product collections.
Identify the Date Range:

Find the maximum (most recent) date in the purchase_date column of orders_df.
Calculate the date three months prior to this maximum date.
Expected Action Result: Two dates identified - the start and end of the last three-month period.
Filter Data for the Last Three Months:

Filter orders_df to include only the rows where purchase_date falls between the start and end dates identified in Step 2.
Expected Action Result: A filtered DataFrame containing only sales data for the last three months.
Sum Quantities by Collection:

Group the filtered DataFrame by the collection column.
Sum the quantity column for each collection.
Expected Action Result: Sum of quantities sold in the last three months for each collection.
Calculate Average Monthly Sales:

Divide the sum obtained in Step 4 by 3 to calculate the average monthly sales for each collection.
Expected Action Result: Average monthly sales for each collection.
Format and Return the Output:

Convert the average monthly sales for each collection into a dictionary format.
Use collection names as keys and the calculated average monthly sales as values.
Expected Output: A JSON object where each key is a product collection name, and each value is the projected sales quantity for that collection for the next month.
Desired Output:

A JSON object (dictionary) with keys as product collection names and values as the projected sales quantity for each collection for the next month, based on the average sales from the last three months.
"""

In [25]:
import pandas as pd
from langchain.llms import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.document_loaders import DataFrameLoader
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [31]:
agent = create_pandas_dataframe_agent(
    OpenAI(temperature=0),
    orders_df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [34]:
result = agent.run(projected_quantity_prompt)



> Entering new AgentExecutor chain...

Thought: I need to access the dataframe, identify the date range, filter the data, sum the quantities by collection, calculate the average monthly sales, and format the output.
Action: python_repl_ast
Action Input: df.head()
Observation:    unique_id             order_id              purchase_date  \
0          1  305-7848953-4909159  2023-01-01T11:03:14+00:00   
1          2  302-4523025-4909942  2023-01-01T11:29:23+00:00   
2          3  028-5725420-6193920  2023-01-01T12:04:47+00:00   
3          4  304-0273788-2526722  2023-01-01T12:06:28+00:00   
4          5  405-7734281-8991534  2023-01-01T13:08:33+00:00   

                 sku               collection      category  quantity  \
0         ayna_gusto       Asymmetric mirrors        Mirror         1   
1         ayna_gusto       Asymmetric mirrors        Mirror         1   
2    ayna_gusto_gold       Asymmetric mirrors        Mirror         1   
3  Berne sehpa 95x65  Baumkante coffee table

In [14]:
prompt = """

Context:
You are an inventory management assistant for an e-commerce brand. Given a product collection, your task is to analyze data and provide useful information for inventory planning and replenishment.

Data Inputs:
inventory_df: DataFrame with current inventory levels, delimited by "<<< >>>": <<<{inventory_df}>>>
orders_df: DataFrame with sales data, delimited by "<<< >>>": <<<{orders_df}>>>
supplying_df: DataFrame with supplier lead times, delimited by "<<< >>>": <<<{supplying_df}>>>
Logistics Days: 16 days

Instructions for Language Model:

Calculate the following metrics, detailing each step taken:

step 1 - Current Inventory Level (current_inventory_level):

Sum the last month's inventory balance for the specified collection in inventory_df.

step 2 - Projected Sales for Next Month (projected_sales_next_month):

Sum the quantity sold for the last three months for the specified collection in orders_df.
Calculate the average and add a 10 percent buffer.

step 3 - Projected Daily Sales (projected_sales_daily):

Divide projected_sales_next_month by 30.

step 4 - Remaining Days of Inventory (remaining_days_of_inventory):

Divide current_inventory_level by projected_sales_daily.

step 5 - Total Replenishment Days (total_replenishment_days):

Retrieve supplier days for the collection from supplying_df.
Add supplier days to logistics days (16 days).

step 6 - Order Placement Day (order_placement_day):

Calculate the day to place an order using remaining_days_of_inventory and total_replenishment_days.

step 7 - Quantity to Order (quantity_to_order):

Calculate to maintain 2 months of inventory, which is twice the projected_sales_next_month.
Provide the calculated values in a dictionary format with the above keys.

Reflect each calculation step taken in bullet points for clarity.

Desired Output:

- A dictionary containing:
current_inventory_level
projected_sales_next_month
projected_sales_daily
remaining_days_of_inventory
total_replenishment_days
order_placement_day
quantity_to_order

- A list of bullet points detailing the calculation steps taken.

"""